So we know from Notebook 10 that trying to make it learn one set of logits per batch is good and gets features right.

How can we make the logits consistent over batches?

Will collect the gradients and gumbel selected values after 5 epochs in each mode. The two modes will be the behavior before burn-in and the behavior after mode-in.

Will follow Notebook 10 with slight modifications (described right before). Remember in Notebook 10, we explored behaviors when all the features were real vs when half the features were noise.

In [1]:
import torch


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#import math

#import gc

from utils import *

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import pearsonr

import seaborn as sns

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
# really good results for vanilla VAE on synthetic data with EPOCHS set to 50, 
# but when running locally set to 10 for reasonable run times
n_epochs = 600
batch_size = 64
lr = 0.0001
b1 = 0.9
b2 = 0.999

In [4]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")
#device = 'cpu'
print("Device")
print(device)

Device
cuda:0


In [5]:
D = 30
N = 10000
z_size = 8

In [6]:
torch.manual_seed(0)
np.random.seed(0)

latent_data = np.random.normal(loc=0.0, scale=1.0, size=N*z_size).reshape(N, z_size)

data_mapper = nn.Sequential(
    nn.Linear(z_size, 2 * z_size, bias=False),
    nn.Tanh(),
    nn.Linear(2 * z_size, D, bias = True),
    nn.ReLU()
).to(device)

data_mapper.requires_grad_(False)

latent_data = Tensor(latent_data)
latent_data.requires_grad_(False)

actual_data = data_mapper(latent_data)


for i in range(5):
    print(torch.sum(actual_data[i,:] != 0))

tensor(14, device='cuda:0')
tensor(18, device='cuda:0')
tensor(14, device='cuda:0')
tensor(17, device='cuda:0')
tensor(16, device='cuda:0')


add noiise

In [7]:
noise_features = torch.empty(N * D).normal_(mean=0,std=0.01).reshape(N, D).to(device)
noise_features.requires_grad_(False)

tensor([[ 0.0013,  0.0135,  0.0054,  ..., -0.0047,  0.0033, -0.0097],
        [ 0.0080, -0.0057,  0.0010,  ...,  0.0009, -0.0134,  0.0105],
        [-0.0103, -0.0029,  0.0185,  ..., -0.0133, -0.0037,  0.0134],
        ...,
        [ 0.0073, -0.0149, -0.0108,  ..., -0.0047, -0.0137,  0.0070],
        [ 0.0006, -0.0141, -0.0124,  ..., -0.0085,  0.0069, -0.0110],
        [-0.0159,  0.0177, -0.0087,  ..., -0.0076, -0.0009,  0.0078]],
       device='cuda:0')

In [8]:
actual_data = torch.cat([actual_data, noise_features], dim = 1)

In [9]:
actual_data.shape

torch.Size([10000, 60])

In [10]:
actual_data = actual_data.cpu().numpy()
scaler = MinMaxScaler()
actual_data = scaler.fit_transform(actual_data)

actual_data = Tensor(actual_data)

slices = np.random.permutation(np.arange(actual_data.shape[0]))
upto = int(.8 * len(actual_data))

train_data = actual_data[slices[:upto]]
test_data = actual_data[slices[upto:]]

In [11]:
global_t = 4

Example of what worked before
Vanilla Gumbel

In [12]:
vae_gumbel_truncated = VAE_Gumbel(2*D, 100, 20, k = 3*z_size, t = global_t)
vae_gumbel_truncated.to(device)
vae_gumbel_trunc_optimizer = torch.optim.Adam(vae_gumbel_truncated.parameters(), 
                                                lr=lr, 
                                                betas = (b1,b2))

In [13]:
gradients_before_burnin = torch.zeros(train_data.shape[1]).to(device)
gradient_post_burn_in = torch.zeros(train_data.shape[1]).to(device)
subset_indices_before_burnin = torch.zeros(train_data.shape[1]).to(device)
subset_indices_post_burnin = torch.zeros(train_data.shape[1]).to(device)

for epoch in range(1, n_epochs+1):
    grads=train_truncated_with_gradients(train_data, vae_gumbel_truncated, 
                                                      vae_gumbel_trunc_optimizer, 
                                                      epoch, 
                                                      batch_size, 
                                                      Dim = 2*D)
    
    vae_gumbel_truncated.t = max(0.001, vae_gumbel_truncated.t * 0.99)
    if epoch <=(n_epochs//5*4):
        gradients_before_burnin += grads
        with torch.no_grad():
            subset_indices_before_burnin += vae_gumbel_truncated.subset_indices.sum(dim = 0)
    if epoch > (n_epochs//5*4):
        gradient_post_burn_in += grads
        with torch.no_grad():
            subset_indices_post_burnin += vae_gumbel_truncated.subset_indices.sum(dim = 0)

Train Epoch: 1 [0/8000 (0%)]	Loss: 42.025238
Train Epoch: 1 [1280/8000 (16%)]	Loss: 41.364944
Train Epoch: 1 [2560/8000 (32%)]	Loss: 40.814362
Train Epoch: 1 [3840/8000 (48%)]	Loss: 40.379112
Train Epoch: 1 [5120/8000 (64%)]	Loss: 39.789692
Train Epoch: 1 [6400/8000 (80%)]	Loss: 39.276009
Train Epoch: 1 [7680/8000 (96%)]	Loss: 38.740120
====> Epoch: 1 Average loss: 40.2651
Train Epoch: 2 [0/8000 (0%)]	Loss: 38.672573
Train Epoch: 2 [1280/8000 (16%)]	Loss: 37.857533
Train Epoch: 2 [2560/8000 (32%)]	Loss: 37.325977
Train Epoch: 2 [3840/8000 (48%)]	Loss: 36.101463
Train Epoch: 2 [5120/8000 (64%)]	Loss: 35.596703
Train Epoch: 2 [6400/8000 (80%)]	Loss: 34.989117
Train Epoch: 2 [7680/8000 (96%)]	Loss: 34.360310
====> Epoch: 2 Average loss: 36.4149
Train Epoch: 3 [0/8000 (0%)]	Loss: 34.523701
Train Epoch: 3 [1280/8000 (16%)]	Loss: 34.206936
Train Epoch: 3 [2560/8000 (32%)]	Loss: 33.856945
Train Epoch: 3 [3840/8000 (48%)]	Loss: 33.793369
Train Epoch: 3 [5120/8000 (64%)]	Loss: 33.988956
Train E

Train Epoch: 22 [6400/8000 (80%)]	Loss: 31.013247
Train Epoch: 22 [7680/8000 (96%)]	Loss: 31.270163
====> Epoch: 22 Average loss: 31.2534
Train Epoch: 23 [0/8000 (0%)]	Loss: 31.400995
Train Epoch: 23 [1280/8000 (16%)]	Loss: 31.041733
Train Epoch: 23 [2560/8000 (32%)]	Loss: 31.345234
Train Epoch: 23 [3840/8000 (48%)]	Loss: 31.320532
Train Epoch: 23 [5120/8000 (64%)]	Loss: 31.026028
Train Epoch: 23 [6400/8000 (80%)]	Loss: 31.138870
Train Epoch: 23 [7680/8000 (96%)]	Loss: 31.540070
====> Epoch: 23 Average loss: 31.1998
Train Epoch: 24 [0/8000 (0%)]	Loss: 31.346783
Train Epoch: 24 [1280/8000 (16%)]	Loss: 31.366404
Train Epoch: 24 [2560/8000 (32%)]	Loss: 31.085634
Train Epoch: 24 [3840/8000 (48%)]	Loss: 31.071608
Train Epoch: 24 [5120/8000 (64%)]	Loss: 31.239252
Train Epoch: 24 [6400/8000 (80%)]	Loss: 30.930275
Train Epoch: 24 [7680/8000 (96%)]	Loss: 30.941076
====> Epoch: 24 Average loss: 31.1411
Train Epoch: 25 [0/8000 (0%)]	Loss: 31.086084
Train Epoch: 25 [1280/8000 (16%)]	Loss: 31.14980

Train Epoch: 44 [1280/8000 (16%)]	Loss: 30.670044
Train Epoch: 44 [2560/8000 (32%)]	Loss: 30.412287
Train Epoch: 44 [3840/8000 (48%)]	Loss: 30.284142
Train Epoch: 44 [5120/8000 (64%)]	Loss: 30.688267
Train Epoch: 44 [6400/8000 (80%)]	Loss: 30.450047
Train Epoch: 44 [7680/8000 (96%)]	Loss: 30.535494
====> Epoch: 44 Average loss: 30.5022
Train Epoch: 45 [0/8000 (0%)]	Loss: 30.533203
Train Epoch: 45 [1280/8000 (16%)]	Loss: 30.846596
Train Epoch: 45 [2560/8000 (32%)]	Loss: 30.508860
Train Epoch: 45 [3840/8000 (48%)]	Loss: 29.904802
Train Epoch: 45 [5120/8000 (64%)]	Loss: 30.310843
Train Epoch: 45 [6400/8000 (80%)]	Loss: 30.303701
Train Epoch: 45 [7680/8000 (96%)]	Loss: 30.523859
====> Epoch: 45 Average loss: 30.4841
Train Epoch: 46 [0/8000 (0%)]	Loss: 30.400795
Train Epoch: 46 [1280/8000 (16%)]	Loss: 30.438272
Train Epoch: 46 [2560/8000 (32%)]	Loss: 30.436558
Train Epoch: 46 [3840/8000 (48%)]	Loss: 30.414251
Train Epoch: 46 [5120/8000 (64%)]	Loss: 30.590418
Train Epoch: 46 [6400/8000 (80%)

Train Epoch: 65 [5120/8000 (64%)]	Loss: 30.314861
Train Epoch: 65 [6400/8000 (80%)]	Loss: 30.275429
Train Epoch: 65 [7680/8000 (96%)]	Loss: 30.515646
====> Epoch: 65 Average loss: 30.2654
Train Epoch: 66 [0/8000 (0%)]	Loss: 30.269190
Train Epoch: 66 [1280/8000 (16%)]	Loss: 30.405798
Train Epoch: 66 [2560/8000 (32%)]	Loss: 30.376183
Train Epoch: 66 [3840/8000 (48%)]	Loss: 30.321278
Train Epoch: 66 [5120/8000 (64%)]	Loss: 30.236647
Train Epoch: 66 [6400/8000 (80%)]	Loss: 30.661760
Train Epoch: 66 [7680/8000 (96%)]	Loss: 30.284760
====> Epoch: 66 Average loss: 30.2637
Train Epoch: 67 [0/8000 (0%)]	Loss: 30.550797
Train Epoch: 67 [1280/8000 (16%)]	Loss: 30.160597
Train Epoch: 67 [2560/8000 (32%)]	Loss: 30.037903
Train Epoch: 67 [3840/8000 (48%)]	Loss: 30.047459
Train Epoch: 67 [5120/8000 (64%)]	Loss: 29.959616
Train Epoch: 67 [6400/8000 (80%)]	Loss: 30.219284
Train Epoch: 67 [7680/8000 (96%)]	Loss: 30.353281
====> Epoch: 67 Average loss: 30.2571
Train Epoch: 68 [0/8000 (0%)]	Loss: 30.20444

Train Epoch: 87 [1280/8000 (16%)]	Loss: 30.178322
Train Epoch: 87 [2560/8000 (32%)]	Loss: 30.091436
Train Epoch: 87 [3840/8000 (48%)]	Loss: 30.255022
Train Epoch: 87 [5120/8000 (64%)]	Loss: 30.174248
Train Epoch: 87 [6400/8000 (80%)]	Loss: 30.241972
Train Epoch: 87 [7680/8000 (96%)]	Loss: 29.892473
====> Epoch: 87 Average loss: 30.1181
Train Epoch: 88 [0/8000 (0%)]	Loss: 29.949902
Train Epoch: 88 [1280/8000 (16%)]	Loss: 30.106182
Train Epoch: 88 [2560/8000 (32%)]	Loss: 30.103258
Train Epoch: 88 [3840/8000 (48%)]	Loss: 29.951073
Train Epoch: 88 [5120/8000 (64%)]	Loss: 30.088888
Train Epoch: 88 [6400/8000 (80%)]	Loss: 29.955593
Train Epoch: 88 [7680/8000 (96%)]	Loss: 29.905472
====> Epoch: 88 Average loss: 30.1153
Train Epoch: 89 [0/8000 (0%)]	Loss: 30.026413
Train Epoch: 89 [1280/8000 (16%)]	Loss: 30.342913
Train Epoch: 89 [2560/8000 (32%)]	Loss: 30.477049
Train Epoch: 89 [3840/8000 (48%)]	Loss: 29.976637
Train Epoch: 89 [5120/8000 (64%)]	Loss: 30.036428
Train Epoch: 89 [6400/8000 (80%)

...

Train Epoch: 171 [3840/8000 (48%)]	Loss: 29.825205
Train Epoch: 171 [5120/8000 (64%)]	Loss: 29.897509
Train Epoch: 171 [6400/8000 (80%)]	Loss: 29.795473
Train Epoch: 171 [7680/8000 (96%)]	Loss: 29.633804
====> Epoch: 171 Average loss: 29.9267
Train Epoch: 172 [0/8000 (0%)]	Loss: 30.154598
Train Epoch: 172 [1280/8000 (16%)]	Loss: 29.911493
Train Epoch: 172 [2560/8000 (32%)]	Loss: 30.012550
Train Epoch: 172 [3840/8000 (48%)]	Loss: 29.933237
Train Epoch: 172 [5120/8000 (64%)]	Loss: 29.937500
Train Epoch: 172 [6400/8000 (80%)]	Loss: 29.816090
Train Epoch: 172 [7680/8000 (96%)]	Loss: 29.972908
====> Epoch: 172 Average loss: 29.9230
Train Epoch: 173 [0/8000 (0%)]	Loss: 29.951574
Train Epoch: 173 [1280/8000 (16%)]	Loss: 30.172197
Train Epoch: 173 [2560/8000 (32%)]	Loss: 29.894226
Train Epoch: 173 [3840/8000 (48%)]	Loss: 30.058176
Train Epoch: 173 [5120/8000 (64%)]	Loss: 30.124512
Train Epoch: 173 [6400/8000 (80%)]	Loss: 29.841183
Train Epoch: 173 [7680/8000 (96%)]	Loss: 29.712990
====> Epoch:

Train Epoch: 192 [3840/8000 (48%)]	Loss: 30.222816
Train Epoch: 192 [5120/8000 (64%)]	Loss: 30.060699
Train Epoch: 192 [6400/8000 (80%)]	Loss: 30.104168
Train Epoch: 192 [7680/8000 (96%)]	Loss: 29.902025
====> Epoch: 192 Average loss: 29.9089
Train Epoch: 193 [0/8000 (0%)]	Loss: 29.930532
Train Epoch: 193 [1280/8000 (16%)]	Loss: 29.928764
Train Epoch: 193 [2560/8000 (32%)]	Loss: 29.963558
Train Epoch: 193 [3840/8000 (48%)]	Loss: 30.271313
Train Epoch: 193 [5120/8000 (64%)]	Loss: 29.973206
Train Epoch: 193 [6400/8000 (80%)]	Loss: 29.941650
Train Epoch: 193 [7680/8000 (96%)]	Loss: 29.890982
====> Epoch: 193 Average loss: 29.9097
Train Epoch: 194 [0/8000 (0%)]	Loss: 30.111902
Train Epoch: 194 [1280/8000 (16%)]	Loss: 29.820795
Train Epoch: 194 [2560/8000 (32%)]	Loss: 29.893639
Train Epoch: 194 [3840/8000 (48%)]	Loss: 29.748861
Train Epoch: 194 [5120/8000 (64%)]	Loss: 29.962934
Train Epoch: 194 [6400/8000 (80%)]	Loss: 30.054029
Train Epoch: 194 [7680/8000 (96%)]	Loss: 30.138664
====> Epoch:

Train Epoch: 213 [3840/8000 (48%)]	Loss: 30.119467
Train Epoch: 213 [5120/8000 (64%)]	Loss: 29.941584
Train Epoch: 213 [6400/8000 (80%)]	Loss: 29.797318
Train Epoch: 213 [7680/8000 (96%)]	Loss: 29.949215
====> Epoch: 213 Average loss: 29.8991
Train Epoch: 214 [0/8000 (0%)]	Loss: 29.839544
Train Epoch: 214 [1280/8000 (16%)]	Loss: 29.883739
Train Epoch: 214 [2560/8000 (32%)]	Loss: 29.887419
Train Epoch: 214 [3840/8000 (48%)]	Loss: 29.690292
Train Epoch: 214 [5120/8000 (64%)]	Loss: 30.246571
Train Epoch: 214 [6400/8000 (80%)]	Loss: 29.978983
Train Epoch: 214 [7680/8000 (96%)]	Loss: 29.981712
====> Epoch: 214 Average loss: 29.9021
Train Epoch: 215 [0/8000 (0%)]	Loss: 29.795774
Train Epoch: 215 [1280/8000 (16%)]	Loss: 30.158306
Train Epoch: 215 [2560/8000 (32%)]	Loss: 29.867483
Train Epoch: 215 [3840/8000 (48%)]	Loss: 29.992905
Train Epoch: 215 [5120/8000 (64%)]	Loss: 29.758713
Train Epoch: 215 [6400/8000 (80%)]	Loss: 30.128817
Train Epoch: 215 [7680/8000 (96%)]	Loss: 29.955023
====> Epoch:

Train Epoch: 234 [3840/8000 (48%)]	Loss: 30.038889
Train Epoch: 234 [5120/8000 (64%)]	Loss: 29.957014
Train Epoch: 234 [6400/8000 (80%)]	Loss: 29.933414
Train Epoch: 234 [7680/8000 (96%)]	Loss: 29.695461
====> Epoch: 234 Average loss: 29.8886
Train Epoch: 235 [0/8000 (0%)]	Loss: 29.780043
Train Epoch: 235 [1280/8000 (16%)]	Loss: 30.145016
Train Epoch: 235 [2560/8000 (32%)]	Loss: 29.855495
Train Epoch: 235 [3840/8000 (48%)]	Loss: 29.794701
Train Epoch: 235 [5120/8000 (64%)]	Loss: 29.772831
Train Epoch: 235 [6400/8000 (80%)]	Loss: 29.978886
Train Epoch: 235 [7680/8000 (96%)]	Loss: 29.753181
====> Epoch: 235 Average loss: 29.8867
Train Epoch: 236 [0/8000 (0%)]	Loss: 29.773815
Train Epoch: 236 [1280/8000 (16%)]	Loss: 29.933281
Train Epoch: 236 [2560/8000 (32%)]	Loss: 29.645767
Train Epoch: 236 [3840/8000 (48%)]	Loss: 29.926254
Train Epoch: 236 [5120/8000 (64%)]	Loss: 29.930956
Train Epoch: 236 [6400/8000 (80%)]	Loss: 29.873896
Train Epoch: 236 [7680/8000 (96%)]	Loss: 30.033524
====> Epoch:

KeyboardInterrupt: 

In [ ]:
print(gradients_before_burnin[:D].sum())
print(gradients_before_burnin[D:].sum())
sns.heatmap(gradients_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])


In [ ]:
print(gradient_post_burn_in[:D].sum())
print(gradient_post_burn_in[D:].sum())
sns.heatmap(gradient_post_burn_in.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_before_burnin[:D].sum())
print(subset_indices_before_burnin[D:].sum())
sns.heatmap(subset_indices_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_post_burnin[:D].sum())
print(subset_indices_post_burnin[(D):].sum())
sns.heatmap(subset_indices_post_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
subset_indices_post_burnin[:(D)].sum() - subset_indices_post_burnin[(D):].sum()

**VAE_Gumbel_NInsta test here**

In [ ]:
vae_gumbel_truncated = VAE_Gumbel_NInsta(2*D, 100, 20, k = 3*z_size, t = global_t)
vae_gumbel_truncated.to(device)
vae_gumbel_trunc_optimizer = torch.optim.Adam(vae_gumbel_truncated.parameters(), 
                                                lr=lr, 
                                                betas = (b1,b2))



In [ ]:
gradients_before_burnin = torch.zeros(train_data.shape[1]).to(device)
gradient_post_burn_in = torch.zeros(train_data.shape[1]).to(device)
subset_indices_before_burnin = torch.zeros(train_data.shape[1]).to(device)
subset_indices_post_burnin = torch.zeros(train_data.shape[1]).to(device)

for epoch in range(1, n_epochs+1):
    grads=train_truncated_with_gradients(train_data, vae_gumbel_truncated, 
                                                      vae_gumbel_trunc_optimizer, 
                                                      epoch, 
                                                      batch_size, 
                                                      Dim = 2*D)
    
    vae_gumbel_truncated.t = max(0.001, vae_gumbel_truncated.t * 0.99)
    if epoch <=(n_epochs//5*4):
        gradients_before_burnin += grads
        with torch.no_grad():
            subset_indices_before_burnin += vae_gumbel_truncated.subset_indices.sum(dim = 0)
    if epoch > (n_epochs//5*4):
        gradient_post_burn_in += grads
        with torch.no_grad():
            subset_indices_post_burnin += vae_gumbel_truncated.subset_indices.sum(dim = 0)

In [ ]:
print(gradients_before_burnin[:D].sum())
print(gradients_before_burnin[D:].sum())
sns.heatmap(gradients_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])


In [ ]:
print(gradient_post_burn_in[:D].sum())
print(gradient_post_burn_in[D:].sum())
sns.heatmap(gradient_post_burn_in.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_before_burnin[:D].sum())
print(subset_indices_before_burnin[D:].sum())
sns.heatmap(subset_indices_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_post_burnin[:D].sum())
print(subset_indices_post_burnin[(D):].sum())
sns.heatmap(subset_indices_post_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
subset_indices_post_burnin[:(D)].sum() - subset_indices_post_burnin[(D):].sum()

The new model


In [ ]:
vae_gumbel_truncated = VAE_Gumbel_NInstaState(2*D, 100, 20, k = 3*z_size, t = global_t)
vae_gumbel_truncated.to(device)
vae_gumbel_trunc_optimizer = torch.optim.Adam(vae_gumbel_truncated.parameters(), 
                                                lr=lr, 
                                                betas = (b1,b2))



In [ ]:
gradients_before_burnin = torch.zeros(train_data.shape[1]).to(device)
gradient_post_burn_in = torch.zeros(train_data.shape[1]).to(device)
subset_indices_before_burnin = torch.zeros(train_data.shape[1]).to(device)
subset_indices_post_burnin = torch.zeros(train_data.shape[1]).to(device)

for epoch in range(1, n_epochs+1):
    grads=train_truncated_with_gradients_gumbel_state(train_data, vae_gumbel_truncated, 
                                                      vae_gumbel_trunc_optimizer, 
                                                      epoch, 
                                                      batch_size, 
                                                      Dim = 2*D)
    
    vae_gumbel_truncated.t = max(0.001, vae_gumbel_truncated.t * 0.99)
    if epoch <=(n_epochs//5*4):
        gradients_before_burnin += grads
        with torch.no_grad():
            subset_indices_before_burnin += sample_subset(vae_gumbel_truncated.logit_enc, 
                                                          vae_gumbel_truncated.k, 
                                                          vae_gumbel_truncated.t).view(-1)
    if epoch == (n_epochs//5*4):
        print("BURN IN DEBUG")
        vae_gumbel_truncated.set_burned_in()
        #vae_gumbel_truncated.t /= 10
        print("Going post burn in")
    if epoch > (n_epochs//5*4):
        gradient_post_burn_in += grads
        with torch.no_grad():
            subset_indices_post_burnin += sample_subset(vae_gumbel_truncated.logit_enc, 
                                                        vae_gumbel_truncated.k, 
                                                        vae_gumbel_truncated.t).view(-1)

In [ ]:
print(gradients_before_burnin[:(D)].mean())
print(gradients_before_burnin[(D):].mean())
sns.heatmap(gradients_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])


In [ ]:
print(gradient_post_burn_in[:D].mean())
print(gradient_post_burn_in[D:].mean())
sns.heatmap(gradient_post_burn_in.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_before_burnin[:D].sum())
print(subset_indices_before_burnin[D:].sum())
sns.heatmap(subset_indices_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_post_burnin[:D].sum())
print(subset_indices_post_burnin[D:].sum())
sns.heatmap(subset_indices_post_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
(subset_indices_post_burnin[:(D)] > 0).sum()

In [ ]:
(subset_indices_post_burnin[(D):] > 0).sum()

In [ ]:
subset_indices_post_burnin[:(D)]

In [ ]:
subset_indices_post_burnin[(D):]

In [ ]:
subset_indices_post_burnin[:(D)].sum() - subset_indices_post_burnin[(D):].sum()

In [ ]:
top_ind = torch.argsort(sample_subset(vae_gumbel_truncated.logit_enc, 
                                                        vae_gumbel_truncated.k, 
                                                        vae_gumbel_truncated.t).view(-1), 
                        descending = True)[:vae_gumbel_truncated.k]

In [ ]:
sum(top_ind < 30)

In [ ]:
sum(top_ind >= 30)